# test af både jupter og det med at importere ting som jeg har tænkt mig


In [2]:
from data.read_binary import readDNSdata
import os
import dask.array as da
import dask
from dask.distributed import Client,as_completed,LocalCluster,get_task_stream
from dask.diagnostics import ResourceProfiler
import xarray as xr


raw_path="../data/raw/"
iterim_path='../data/interim/'






In [4]:
cluster=LocalCluster(n_workers=2,)
client =Client(cluster)

C:\Users\thor\anaconda3\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57621 instead
  warnings.warn(


In [5]:
filelist = os.listdir(raw_path)
filelist=sorted(filelist)


filelist_raw=[]

for i in range(len(filelist)):
    filelist_raw.append(raw_path+filelist[i])



In [20]:
futures=client.map(readDNSdata,filelist_raw,key=filelist)
for future, [quantities, quanList, xF, yF, zF, length, storl, paraString] in as_completed(futures, with_results=True):
    xF=xF[:-1]
    zF=zF[:-1]
    ds=xr.Dataset(
        {
            "u_vel": (['x','y','z'],quantities[0]),
            "v_vel": (['x','y','z'],quantities[1]),
            "w_vel": (['x','y','z'],quantities[2]),
            "pr1": (['x','y','z'],quantities[3]),
            "pr0.71": (['x','y','z'],quantities[4]),
            "pr0.2": (['x','y','z'],quantities[5]),
            "pr0.025": (['x','y','z'],quantities[6]),
        },
        coords={
            "x":(["x"],xF),
            "y":(["y"],yF),
            "z":(["z"],zF),
            "time":length[2],
        },

    ).chunk(1000000)
    ds.to_netcdf(future.key[:-1]+'nc')

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

Det eneste jeg skal have gjort nu er at lave det her til en funktion, og få den til at kopiere dataen hen til min scratch før den begynder at regne, og kopiere det tilbage bagefter
+ fået den til at køre med slurm lol
+ kompression
